# Naive Bayesian Classifier
### Q1. Bayes Rule을 이해하고 Naive  Bayes classifier가 사용하는 사후 확률 계산 과정을 서술하세요.

- Bayes Rule:   
$P(w_i|x) = \frac{P(x|w_i)|P(w_i)}{P(x)} = \frac{P(x|w_i) P(w_i)}{\Sigma_j P(x|w_j)P(w_j)}$
  -
  - $P(x|w_i)\text{: 사후 확률, posterior}\\
P(x|w_i) \text{: 가능도/우도, likelihood}\\
P(w_i) \text{: 사전 확률, prior}\\
P(x) \text{: 증거, evidence}$

A1.
특성 간의 독립성을 가정하여 계산을 단순화함
사전확률과 가능도를 계산하고, 이를 기반으로 사후확률을 계산하여 가장 높은 확률을 가진 클래스를 예측하는 것임



### Q2. Naive Bayes Classification 방법을 이용해서 다음 생성된 리뷰 데이터에 기반한 감정 분석을 해봅시다.

In [22]:
# pip install pandas
import pandas as pd
import re

In [23]:
# 리뷰 데이터 생성
data = {
    'review': [
        'I love this great product! It exceeded my expectations.',
        'The Worst purchase I have ever made. Completely useless.',
        'It is an average product, nothing special but not terrible either.',
        'Great service and who can help but love this design? Highly recommend!',
        'Terrible experience, I will never buy from this poor brand again.',
        'It’s acceptable, but I expected better service, not just an acceptable one.',
        'Absolutely wonderful! I am very satisfied with this great service.',
        'The quality is poor and it broke after one use. Terrible enough!',
        'Acceptable product for the price, but there are better options out there.',
        'Great quality and fast shipping with wonderful service! I love it'
    ],
    'sentiment': [
        'positive', 'negative', 'neutral', 'positive', 'negative',
        'neutral', 'positive', 'negative', 'neutral', 'positive',
    ]
}
df = pd.DataFrame(data)
df.head()

,review,sentiment
0,I love this great product! It exceeded my expe...,positive
1,The Worst purchase I have ever made. Completel...,negative
2,"It is an average product, nothing special but ...",neutral
3,Great service and who can help but love this d...,positive
4,"Terrible experience, I will never buy from thi...",negative


In [24]:
# 불용어 리스트 정의
stopwords = ['i', 'my', 'am', 'this', 'it', 'its', 'an', 'a', 'the', 'is', 'are', 'and', 'product', 'service']

In [25]:
# 텍스트 전처리 함수 정의
def preprocess_text(text):
    # 소문자로 변환
    text = text.lower()
    # 특수 기호 제거
    text = re.sub(r'[^a-z\s]', '', text)
    # 불용어 제거
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# 모든 리뷰에 대해 전처리 수행
df['review'] = df['review'].apply(preprocess_text)

기본적인 데이터 전처리가 완료되었습니다!
이제부터 직접 나이브 베이지안 분류를 수행해 봅시다.  
우리가 분류하고자 하는 문장은 총 두가지 입니다.  
전처리가 완료되었다고 치고,   
첫번째 문장은 **'love, great, awesome'**,  
두번째 문장은 **'terrible, not, never'** 입니다.

사전 확률 $P(positive), P(negative), P(neutral)$을 구합니다.

In [26]:
# 사전 확률 계산 함수
def calculate_prior_probabilities(df):
    total_count = len(df)
    class_counts = df['sentiment'].value_counts()

    # 사전 확률 계산
    prior_probabilities = {
        sentiment: count / total_count
        for sentiment, count in class_counts.items()
    }

    return prior_probabilities

# 사전 확률 계산
prior_probabilities = calculate_prior_probabilities(df)
print("사전 확률:")
for sentiment, prob in prior_probabilities.items():
    print(f"P({sentiment}): {prob}")


사전 확률:
P(positive): 0.4
P(negative): 0.3
P(neutral): 0.3


가능도를 구하기 위한 확률들을 계산합니다.  
예: 첫번째 문장 분류를 위해서는, $P(love|positive), P(great|positive), P(awesome|positive)\\
P(love|negative), P(great|negative), P(awesome|negative)\\
P(love|neutral), P(great|neutral), P(great|neutral)$를 구합니다.

이 때 CountVectorizer를 사용하여 도출한 단어 벡터를 활용하면 확률들을 간편하게 구할 수 있습니다.  
참고: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()
review_array

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [34]:
feature_names = vectorizer.get_feature_names_out()

In [35]:
vectorizer.vocabulary_

{'love': 28,
 'great': 23,
 'exceeded': 16,
 'expectations': 17,
 'worst': 53,
 'purchase': 38,
 'have': 24,
 'ever': 15,
 'made': 29,
 'completely': 11,
 'useless': 47,
 'average': 4,
 'nothing': 32,
 'special': 43,
 'but': 8,
 'not': 31,
 'terrible': 44,
 'either': 13,
 'who': 49,
 'can': 10,
 'help': 25,
 'design': 12,
 'highly': 26,
 'recommend': 40,
 'experience': 19,
 'will': 50,
 'never': 30,
 'buy': 9,
 'from': 22,
 'poor': 36,
 'brand': 6,
 'again': 3,
 'acceptable': 1,
 'expected': 18,
 'better': 5,
 'just': 27,
 'one': 33,
 'absolutely': 0,
 'wonderful': 52,
 'very': 48,
 'satisfied': 41,
 'with': 51,
 'quality': 39,
 'broke': 7,
 'after': 2,
 'use': 46,
 'enough': 14,
 'for': 21,
 'price': 37,
 'there': 45,
 'options': 34,
 'out': 35,
 'fast': 20,
 'shipping': 42}

In [36]:
frequency_matrix = pd.DataFrame(review_array, columns = feature_names)
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)
frequency_matrix

,sentiment,absolutely,acceptable,after,again,average,better,brand,broke,but,...,terrible,there,use,useless,very,who,will,with,wonderful,worst
0,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,negative,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,neutral,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,positive,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,negative,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
5,neutral,0,2,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,positive,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
7,negative,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
8,neutral,0,1,0,0,0,1,0,0,1,...,0,2,0,0,0,0,0,0,0,0
9,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [43]:
# 각 클래스별 단어 빈도수 계산
def class_word_counts(df, class_label):
    class_df = df[df['sentiment'] == class_label]
    return class_df[feature_names].sum()

# 각 클래스의 총 단어 수 계산
def total_word_count(df, class_label):
    class_df = df[df['sentiment'] == class_label]
    return class_df[feature_names].sum().sum()

# 조건부 확률 계산
def conditional_probabilities(df, class_label):
    word_counts = class_word_counts(df, class_label)
    total_count = total_word_count(df, class_label)
    # Laplace smoothing
    return (word_counts + 1) / (total_count + len(feature_names))

# 각 클래스의 조건부 확률 계산
class_labels = ['positive', 'negative', 'neutral']
conditional_probs = {label: conditional_probabilities(df, label) for label in class_labels}

KeyError: "None of [Index(['absolutely', 'acceptable', 'after', 'again', 'average', 'better',\n       'brand', 'broke', 'but', 'buy', 'can', 'completely', 'design', 'either',\n       'enough', 'ever', 'exceeded', 'expectations', 'expected', 'experience',\n       'fast', 'for', 'from', 'great', 'have', 'help', 'highly', 'just',\n       'love', 'made', 'never', 'not', 'nothing', 'one', 'options', 'out',\n       'poor', 'price', 'purchase', 'quality', 'recommend', 'satisfied',\n       'shipping', 'special', 'terrible', 'there', 'use', 'useless', 'very',\n       'who', 'will', 'with', 'wonderful', 'worst'],\n      dtype='object')] are in the [columns]"

독립성 가정을 이용하여 가능도(likelihood)를 구합니다.  
첫번째 문장 예시: $P(love, great, awesome|positive), P(love, great, awesome|negative), P(love, great, awesome|neutral)$

In [42]:
# 가능도 구하는 코드를 작성해주세요.
# 가능도(likelihood) 계산
def calculate_likelihood(words, class_label, conditional_probs):
    probs = [conditional_probs[class_label].get(word, 0) for word in words]
    likelihood = 1
    for prob in probs:
        likelihood *= prob
    return likelihood

# 예시 문장
target_review1 = ['love', 'great', 'awesome']
target_review2 = ['terrible', 'not', 'never']

# 각 문장에 대한 가능도 계산
def print_likelihoods(target_review, class_labels, conditional_probs):
    print(f"Likelihoods for {target_review}:")
    for label in class_labels:
        likelihood = calculate_likelihood(target_review, label, conditional_probs)
        print(f"Likelihood for '{label}': {likelihood}")

print_likelihoods(target_review1, class_labels, conditional_probs)
print_likelihoods(target_review2, class_labels, conditional_probs)

NameError: name 'conditional_probs' is not defined

위에서 구한 사전 확률과 가능도를 이용하여 타겟 문장이 positive, negative, neutral일 확률을 구하고 최종적으로 어떤 감성일지 분석해봅니다.

In [44]:
import numpy as np
# 최종 확률 구하는 코드를 작성해주세요.
# 첫번째 문장
# P(positive|target_review1)

# P(negative|target_review1)

# P(neutral|target_review1)

# 두번째 문장
# P(positive|target_review2)

# P(negative|target_review2)

# P(neutral|target_review2)


posterior1 = calculate_posterior(target_review1, class_labels, conditional_probs, prior_probs)
posterior2 = calculate_posterior(target_review2, class_labels, conditional_probs, prior_probs)

print(f"Posterior probabilities for '{target_review1}':")
for label, prob in posterior1.items():
    print(f"P({label}|target_review1): {prob:.4f}")

print(f"Posterior probabilities for '{target_review2}':")
for label, prob in posterior2.items():
    print(f"P({label}|target_review2): {prob:.4f}")

NameError: name 'calculate_posterior' is not defined

A2-1.   
Target review1의 분류 결과:   
Target review2의 분류 결과:

Q2-2. 나이브 베이지안 기반 확률을 구하는 과정에서 어떤 문제점을 발견할 수 있었나요? 그리고 그 문제를 해결하기 위한 방법에 대해 간략하게 조사 및 서술해 주세요. (힌트: Laplace smoothing)

A2-2. 단어의 희소성 문제 - 훈련데이터에 특정 단어가 포함되지 않는 경우, 해당 단어의 조건부 확률은 0으로 계산됨

Laplace smoothing - 모든 단어에 대해 비제로 확률 보장, 일반화 성능이 향상되며, 드물게 나타나는 단어들에 대한 예측이 보다 현실적임